In [1]:

import SimpleITK as sitk

#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, fixed
import os
import gui
import resample_itk_image
from myshow import myshow, myshow3d
from itertools import chain

import pandas as pd

%matplotlib widget

In [2]:
# options
plot_values_hist = False
save_seeds = False
save_output = False
acquire_points = False
use_seeds = False
load_seed = False
use_threshold_only = True

# lower and upper intensity thresholds for segmentation 
lower_threshold = 500
upper_threshold = 1800


In [17]:
# PC
data_folder = r"C:\Users\david\UW\Ryan Kellogg - Kempe files\subject_data"
data_folder = os.path.abspath(os.path.expanduser(os.path.expandvars(data_folder)))

In [18]:
# create output directories if they don't exist

#for root, dirs, files in os.walk(data_folder, topdown=True):
    # walking tips from https://stackoverflow.com/questions/42720627/python-os-walk-to-certain-level
#    if root[len(data_folder):].count(os.sep) == 2:
#        if not (os.path.exists(os.path.join(root,'output'))):
#            os.mkdir(os.path.join(root,'output'))

In [1]:
# generate structure of files 

data_subj_list = list()
output_dir_list = list()
for data_subj, dirs, files in os.walk(data_folder, topdown=True):
    # walking tips from https://stackoverflow.com/questions/42720627/python-os-walk-to-certain-level
    if (data_subj[len(data_folder):].count(os.sep) == 3) and not 'output' in data_subj[len(data_folder):]:
  #      print(data_subj)
        super_folder = data_subj.rsplit('\\',1)
        OUTPUT_DIR = os.path.join(super_folder[0],'output')
        
        data_subj_list.append(data_subj)
        output_dir_list.append(OUTPUT_DIR)

length_data = len(data_subj_list)

NameError: name 'os' is not defined

In [23]:
select = 0

In [24]:
# Read the original series. First obtain the series file names using the
# image series reader.
data_subj = data_subj_list[select]
OUTPUT_DIR = output_dir_list[select]
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_subj)
if not series_IDs:
    print("ERROR: given directory \""+data_folder+"\" does not contain a DICOM series.")

series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_subj, series_IDs[0])

series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

# Configure the reader to load all of the DICOM tags (public+private):
# By default tags are not loaded (saves time).
# By default if tags are loaded, the private tags are not loaded.
# We explicitly configure the reader to load tags, including the
# private ones.
series_reader.MetaDataDictionaryArrayUpdateOn()
#series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()

reader = sitk.ImageFileReader()
reader.SetFileName(series_file_names[0])
#reader.LoadPrivateTagsOn();

reader.ReadImageInformation();

#for k in reader.GetMetaDataKeys():
#    v = reader.GetMetaData(k)
#    print("({0}) = = \"{1}\"".format(k,v))

print('Original Image Spacing ',image3D.GetSpacing())
print('Original Image Size ', image3D.GetSize())

orig_size = image3D.GetSize()
orig_spacing = image3D.GetSpacing()
min_orig = min(orig_spacing)
max_orig = max(orig_spacing)

image3D_resample = resample_itk_image.resample_img(image3D,[max_orig,max_orig,max_orig])

print('Resampled Image Spacing ',image3D_resample.GetSpacing())
print('Resampled Image Size ', image3D_resample.GetSize())

image3D_resample_seq = resample_itk_image.resample_img(image3D,[min_orig,min_orig,min_orig])

print('Resampled Image Spacing For Sequential ',image3D_resample_seq.GetSpacing())
print('Resampled Image Size For Sequential', image3D_resample_seq.GetSize())

# recast types as necessary for analysis 

recast_image3D = sitk.Cast(sitk.RescaleIntensity(image3D_resample), sitk.sitkUInt8)
recast32_image3D = sitk.Cast(image3D_resample, sitk.sitkFloat32)
recast32_image3D_seq = sitk.Cast(image3D_resample_seq, sitk.sitkFloat32)
recastUint8_image3D_seq = sitk.Cast(image3D_resample_seq, sitk.sitkUInt8)

#
#img = sitk.ReadImage(series_file_names)

# plot halfway 
#z = int(image3D.GetDepth()/2)
#plt.imshow(sitk.GetArrayViewFromImage(image3D)[z,:,:], cmap=plt.cm.Greys_r)
#plt.axis('off');

intensity_values = sitk.GetArrayViewFromImage(image3D)
values_list = list(intensity_values)
values_flattened = list(chain.from_iterable(values_list))
values_flattened = np.array(values_flattened).flatten()

if plot_values_hist:
    plt.figure()
    plt.hist(values_flattened, bins=100)
    plt.title("Intensity Values in ROI")
    plt.show()      

Original Image Spacing  (0.369140625, 0.36927621861152, 2.000081777572632)
Original Image Size  (512, 677, 108)
Resampled Image Spacing  (2.000081777572632, 2.000081777572632, 2.000081777572632)
Resampled Image Size  (94, 125, 108)
Resampled Image Spacing For Sequential  (0.369140625, 0.369140625, 0.369140625)
Resampled Image Size For Sequential (512, 677, 585)


In [25]:
# try histogram 
if use_threshold_only:
    seg = sitk.BinaryThreshold(image3D_resample_seq, lowerThreshold=lower_threshold, upperThreshold=upper_threshold, insideValue=1, outsideValue=0)
    
   # vector_radius_open=(1,1,1)
    vector_radius_close=(3,3,3)
    kernel=sitk.sitkBall
    seg_resized_clean = sitk.BinaryMorphologicalClosing(seg, vector_radius_close, kernel)
  #  seg_resized_clean = sitk.BinaryMorphologicalOpening(seg_resized_clean, vector_radius_open, kernel)

    seg_resized_clean = resample_itk_image.resample_img_set_back_orig(seg_resized_clean,orig_spacing,orig_size,True)
    
    seg_resized = resample_itk_image.resample_img_set_back_orig(seg,orig_spacing,orig_size,True)
    
    connected = sitk.ConnectedComponent(seg_resized_clean)
    connected = sitk.RelabelComponent(connected,sortByObjectSize=True)
    connected_select = connected == 1
    connected_resized = resample_itk_image.resample_img_set_back_orig(connected_select,orig_spacing,orig_size,True)

    


    #seg = sitk.Image(image3D_resample.GetSize(), sitk.sitkUInt8)
#seg.CopyInformation(image3D_resample)


In [26]:
myshow3d(image3D)

interactive(children=(IntSlider(value=53, description='z', max=107), Output()), _dom_classes=('widget-interact…

In [27]:
myshow3d(connected_select)


interactive(children=(IntSlider(value=53, description='z', max=107), Output()), _dom_classes=('widget-interact…

In [62]:
# select points
if acquire_points and use_seeds:
    point_acquisition_interface = gui.PointDataAquisition(image3D_resample)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
if acquire_points and use_seeds:
    initial_seed_point_indexes = point_acquisition_interface.get_point_indexes()

    # write points used to file for validation 
    initial_seed_point_indexes
    if save_seeds:
        np.savetxt(os.path.join(OUTPUT_DIR,'seeds.csv'),initial_seed_point_indexes,delimiter=',')


In [64]:
if load_seed and use_seeds:
    initial_seed_point_indexes = np.loadtxt(os.path.join(OUTPUT_DIR,'seeds.csv'),delimiter=',')
    initial_seed_point_indexes = initial_seed_point_indexes.astype(int)
    initial_seed_point_indexes = initial_seed_point_indexes.tolist()

In [65]:
# implicit thresholds 

#seg_implicit_thresholds = sitk.ConfidenceConnected(image3D, seedList=initial_seed_point_indexes,
#                                                   numberOfIterations=1,
#                                                   multiplier=2,
#                                                   initialNeighborhoodRadius=5,
#                                                   replaceValue=1)

#seg_implicit_thresholds = sitk.ConnectedThreshold(image3D_resample, seedList=initial_seed_point_indexes, lower=400, upper=1700)

#gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(recast_image3D, seg_implicit_thresholds)],title_list = ['confidence connected result'])

#myshow3d(seg_implicit_thresholds)
if use_seeds:
    seg = sitk.Image(image3D_resample.GetSize(), sitk.sitkUInt8)
    seg.CopyInformation(image3D_resample)
    for index in range(1,len(initial_seed_point_indexes)):
        seg[initial_seed_point_indexes[index]] = 1
    seg = sitk.BinaryDilate(seg, 10)

    myshow(sitk.LabelOverlay(recast_image3D, seg))

interactive(children=(IntSlider(value=53, description='z', max=107), Output()), _dom_classes=('widget-interact…

In [66]:
#stats = sitk.LabelStatisticsImageFilter()
#stats.Execute(image3D, seg)

#factor = 3.5
#lower_threshold = stats.GetMean(1)-factor*stats.GetSigma(1)
#upper_threshold = stats.GetMean(1)+factor*stats.GetSigma(1)
#print(stats)


# intialize distance map
if use_seeds:
    init_ls = sitk.SignedMaurerDistanceMap(seg, insideIsPositive=True, useImageSpacing=True)

    # initialize threshold segmentation level set filter
    lsFilter = sitk.ThresholdSegmentationLevelSetImageFilter()
    lsFilter.SetLowerThreshold(lower_threshold)
    lsFilter.SetUpperThreshold(upper_threshold)
    lsFilter.SetMaximumRMSError(0.01)
    lsFilter.SetNumberOfIterations(1000)
    lsFilter.SetCurvatureScaling(1)
    lsFilter.SetPropagationScaling(1)
    lsFilter.ReverseExpansionDirectionOn()

    #niter = 0
    #for i in range(0,10):
    #    ls = lsFilter.Execute(init_ls,recast32_image3D)
    #    niter += lsFilter.GetNumberOfIterations()
    #    t = "LevelSet after "+str(niter)+" iterations and RMS "+str(lsFilter.GetRMSChange())
    #    fig = myshow(sitk.LabelOverlay(recast_image3D, ls > 0),title=t)

    # run filter
    ls = lsFilter.Execute(init_ls,recast32_image3D)    

    # print results 
    print(lsFilter)

    #myshow(sitk.LabelOverlay(recast_image3D, ls>0))
    # extract >0 values for segmentation 
    ls_thresh = ls>0

    image3D_resample_seq_intermed = resample_itk_image.resample_img_set_back_orig(ls_thresh,orig_spacing,orig_size,True)
    image3D_resample_seq = resample_itk_image.resample_img(image3D_resample_seq_intermed,[min_orig,min_orig,min_orig],True)

In [45]:
# intialize distance map
if use_seeds:
    init_ls_seq = sitk.SignedMaurerDistanceMap(image3D_resample_seq, insideIsPositive=True, useImageSpacing=True)
elif use_threshold_only:
    init_ls_seq = sitk.SignedMaurerDistanceMap(seg, insideIsPositive=True, useImageSpacing=True)

# initialize threshold segmentation level set filter
lsFilterSeq = sitk.ThresholdSegmentationLevelSetImageFilter()
lsFilterSeq.SetLowerThreshold(lower_threshold)
lsFilterSeq.SetUpperThreshold(upper_threshold)
lsFilterSeq.SetMaximumRMSError(0.005)
lsFilterSeq.SetNumberOfIterations(1000)
lsFilterSeq.SetCurvatureScaling(1)
lsFilterSeq.SetPropagationScaling(1)
lsFilterSeq.ReverseExpansionDirectionOn()

# run filter
lsSeq = lsFilterSeq.Execute(init_ls_seq,recast32_image3D_seq)    

# print results 
print(lsFilterSeq)

#myshow(sitk.LabelOverlay(recast_image3D, ls>0))
# extract >0 values for segmentation 
ls_thresh_seq = lsSeq>0

itk::simple::ThresholdSegmentationLevelSetImageFilter
  LowerThreshold: 500
  UpperThreshold: 1800
  MaximumRMSError: 0.005
  PropagationScaling: 1
  CurvatureScaling: 1
  NumberOfIterations: 1000
  ReverseExpansionDirection: 1
  ElapsedIterations: 17
  RMSChange: 0.00492387
  Debug: 0
  NumberOfThreads: 12
  Commands: (none)
  ProgressMeasurement: 0.017
  ActiveProcess: (none)



In [130]:
myshow3d(ls_thresh_seq)

ls_thresh_resize = resample_itk_image.resample_img_set_back_orig(ls_thresh_seq,orig_spacing,orig_size,True)
recast_ls = sitk.Cast(ls_thresh_resize,sitk.sitkUInt8)

print('Final Image Spacing ',ls_thresh_resize.GetSpacing())
print('Final Image Size ', ls_thresh_resize.GetSize())

#shape_stats = sitk.LabelShapeStatisticsImageFilter()
#shape_stats.ComputeOrientedBoundingBoxOn()
#shape_stats.Execute(recast_ls)
#shape_stats.GetPhysicalSize(1)

# Write the image.
if save_output:
    if use_seeds:
        output_file_name_3D = os.path.join(OUTPUT_DIR, 'segmentation_threshold_only.nii')
        sitk.WriteImage(ls_thresh_resize, output_file_name_3D)   
        
    if use_threshold_only:
        output_file_name_3D = os.path.join(OUTPUT_DIR, 'segmentation_threshold_only.nii')
        sitk.WriteImage(ls_thresh_resize, output_file_name_3D)

        output_file_name_3D = os.path.join(OUTPUT_DIR, 'segmentation_threshold_only_seg.nii')
        sitk.WriteImage(seg_resized, output_file_name_3D)

        output_file_name_3D = os.path.join(OUTPUT_DIR, 'segmentation_threshold.nii')
        sitk.WriteImage(seg_resized_clean, output_file_name_3D)
        
        output_file_name_3D = os.path.join(OUTPUT_DIR, 'segmentation_threshold_connected.nii')
        sitk.WriteImage(seg_resized_clean_connected, output_file_name_3D)

interactive(children=(IntSlider(value=292, description='z', max=584), Output()), _dom_classes=('widget-interac…

Final Image Spacing  (0.369140625, 0.36927621861152, 2.000081777572632)
Final Image Size  (512, 677, 108)
